# PSTAT 134 HW 2
##  Shon Inouye

### Problem 1: Data Download

In [ ]:
# Import data
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # set plot style
import pandas as pd
import numpy as np
from ipywidgets import interact, FloatSlider, Dropdown, Button

In [ ]:
# Define function to pull nba data
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(pd.DataFrame(d, columns=h))

In [ ]:
## get all teams
params = {'LeagueID':'00'}
teams = get_nba_data('commonTeamYears', params)

## get all players
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = get_nba_data('commonallplayers', params)

## get all team franchise history
params = {'LeagueID':'00'}
history = get_nba_data('franchisehistory',params)

In [ ]:
# Change type of columns for teams
teams.ABBREVIATION = teams.ABBREVIATION.astype('category')
teams.TEAM_ID      = teams.TEAM_ID.astype('category')
teams.MIN_YEAR     = teams.MIN_YEAR.astype('int')
teams.MAX_YEAR     = teams.MAX_YEAR.astype('int')

In [ ]:
# Subset the current teams
teams = teams[teams.MAX_YEAR == 2017]
teams['TEAM_AGE'] = teams.MAX_YEAR - teams.MIN_YEAR

teams_clean = teams.copy() ## make a copy for later

In [ ]:
teams.head()

In [ ]:
# Subset just the players in current teams
players = players[players.TEAM_ID.isin(teams.TEAM_ID)]

In [ ]:
players.head()

In [ ]:
# Create a table of unique rows with full team names
team_names = players[['TEAM_ABBREVIATION', 'TEAM_CODE']].drop_duplicates()#.set_index('TEAM_ABBREVIATION')
team_names.head()

In [ ]:
# Add team codes as a new column
teams = pd.merge(teams_clean, team_names, left_on='ABBREVIATION', right_on='TEAM_ABBREVIATION')

teams.TEAM_CODE = teams.TEAM_CODE.str.capitalize() # returns values so needs to be reassigned
teams.sort_values('ABBREVIATION', inplace=True)    # modifies object
teams.tail()

In [ ]:
# Create new column for year range
history['YEAR_RANGE'] = history.START_YEAR+' - '+history.END_YEAR
history.sort_values('TEAM_ID', inplace=True)
history.head()

### Problem 2: Creating interactive widgets

#### Widget 1

In [ ]:
menu = {
    'juice':['apple', 'peach', 'grape'],
    'tea':['ginger', 'green', 'earl grey'],
}

selected1 = 'tea'

flavor = Dropdown(options=menu[selected1], value=menu[selected1][0])
drink = Dropdown(options=menu.keys(), value=selected1)

def update_drink(change):
    flavor.index = None
    flavor.options = menu[change['new']]
    flavor.selected = menu[change['new']][0]

    
drink.observe(update_drink, names='value')
 
display(flavor, drink)

#### Widget 2

In [ ]:
team_dd_text = teams.TEAM_ABBREVIATION+', '+teams.TEAM_CODE
team_dd = dict(zip(team_dd_text, teams.TEAM_ID))

plyr_by_team_dd = dict()

for t, p in players.groupby('TEAM_ID'):
    plyr_by_team_dd[t] = dict(zip(p.DISPLAY_LAST_COMMA_FIRST, p.PERSON_ID))

In [ ]:
selected2 = 'LAC, Clippers'

team_menu1 = Dropdown(options=team_dd, label=selected2)
plyr_menu1 = Dropdown(options=plyr_by_team_dd[team_dd[selected2]])

display(team_menu1, plyr_menu1)

## update players list
def update_team1(change):
    plyr_menu1.index = None
    plyr_menu1.options = plyr_by_team_dd[change['new']]

team_menu1.observe(update_team, names='value')



#### Widget 3

In [ ]:
team_dd_text2 = history.TEAM_CITY+' '+history.TEAM_NAME
team_dd2 = dict(zip(team_dd_text2, history.TEAM_ID))

In [ ]:
hist_by_team_dd = dict()

for t, h in history.groupby('TEAM_ID'):
    
    hist_by_team_dd[t] = dict(zip(h.YEAR_RANGE, h.GAMES))


In [ ]:
selected3 = 'Atlanta Hawks'

team_menu2 = Dropdown(options=team_dd2, label=selected3)
hist_menu = Dropdown(options=hist_by_team_dd[team_dd2[selected3]])

display(team_menu2, hist_menu)

def update_team2(change):
    hist_menu.options = hist_by_team_dd[change['new']]
    hist_menu.selected = hist_by_team_dd[change['new']]

team_menu2.observe(update_team2, names='value')

### Problem 3: Downloading data with changing widget states

#### Widget 1

In [ ]:
order = Button(description='Order!', icon='check')

def make_order(change):
    print(flavor.value, drink.value)
    
order.on_click(make_order)
 
display(flavor, drink, order)

#### Widget 2

In [ ]:
fetch_button1 = Button(description='Get Data!', icon='check')

display(team_menu1, plyr_menu1, fetch_button1)

## get data action
def get_data1(change):
    print(team_menu1.value, plyr_menu1.value)
    
fetch_button1.on_click(get_data1)

#### Widget 3

In [ ]:
fetch_button2 = Button(description='Get Data!', icon='check')

display(team_menu2, hist_menu, fetch_button2)

## update players list
def update_team(change):
    hist_menu.options = hist_by_team_dd[change['new']]
    #team_menu.options = team_dd[change['new']]

team_menu.observe(update_team2, names='value')

## get data action
def get_data2(change):
    print('Games:',hist_menu.value)
    
fetch_button2.on_click(get_data2)

### Problem 4: Data transformation and visualization

In [ ]:
# Feature engineering
history.START_YEAR = history.START_YEAR.astype('int')
history.END_YEAR = history.END_YEAR.astype('int')
history.END_YEAR = history.END_YEAR.astype('int')
history['FULL_NAME'] = history.TEAM_CITY+' '+history.TEAM_NAME
history['AGE'] = history.END_YEAR - history.START_YEAR

history.head()

In [ ]:
# Take average number of games played for number of league titles
leagueTitles = history.groupby('LEAGUE_TITLES')['GAMES'].mean()
leagueTitles

In [ ]:
# Plot data
leagueTitles_Games.plot.bar()
plt.xlabel('# of League Titles Won')
plt.ylabel('Average Number of Games Played')
plt.title('Average Games Played per League Titles Won')
plt.show()

We can see that as the number of league titles a team wins increases, so does the average number of games played. This makes sense since more games played results in more experience.

In [ ]:
# Take average number of games played for number of league titles
leagueTitles_WinPct = history.groupby('LEAGUE_TITLES')['WIN_PCT'].mean()
leagueTitles_WinPct

In [ ]:
leagueTitles_WinPct.plot.bar()
plt.xlabel('# of League Titles Won')
plt.ylabel('Average Win Percentage')
plt.title('Average Win Percentage per League Titles Won')
plt.show()

Similar to the previous graph, we can see that as the number of league titles a team wins increases, so does the win percentage. 